In [1]:
from PIL import Image
from numpy import *
from scipy.ndimage import filters
import sys
import imageio
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def edge_strength(input_image):
    grayscale = array(input_image.convert('L'))
    filtered_y = zeros(grayscale.shape)
    filters.sobel(grayscale,0,filtered_y)
    return sqrt(filtered_y**2)

In [3]:
def draw_edge(image, y_coordinates, color, thickness):
    for (x, y) in enumerate(y_coordinates):
        for t in range( int(max(y-int(thickness/2), 0)), int(min(y+int(thickness/2), image.size[1]-1 )) ):
            image.putpixel((x, t), color)
    return image

In [4]:
gt_row = -1
gt_col = -1
input_filename = "test_images/mountain.jpg"
input_image = Image.open(input_filename)

In [7]:
image1 = imageio.imread(input_filename)
image1.shape

(141, 251, 3)

In [11]:
edge_strength = edge_strength(input_image)
df = pd.DataFrame(data=edge_strength, index=None)

In [90]:
edge_strength.shape

(141, 251)

In [12]:
df.shape

(141, 251)

In [76]:
trans_p = [0.5, 0.2545, 0.005]
for i in range(0,df.shape[0]):
    trans_p[i] = 0.5 - i*5/df.shape[0]

In [93]:
t_strength_col = zeros(df.shape[1])
for i in range(df.shape[1]):
    for j in range(df.shape[0]):
        t_strength_col[i] += edge_strength[j][i]
t_strength_col[0]

4240.0

In [87]:
# t_strength_col
emi_prob = zeros((df.shape[0],df.shape[1]))
for row in range(df.shape[0]):
    emi_prob[row][0] = edge_strength[row][0]/t_strength_col[0]    

In [79]:
store_states = zeros((df.shape[0],df.shape[1]))

In [95]:
for col in range(1, df.shape[1]):
    for row in range(df.shape[0]):
        max_search = 0
        for j in range(-2,3):
            if row + j < df.shape[0] & row + j >= 0:
                if max_search < emi_prob[row + j][col - 1] * trans_p[abs(j)]:
                    max_search = emi_prob[row + j][col - 1] * trans_p[abs(j)]
                    print(max_search)
                    print((row,col))
                    store_states[row][col] = row + j
                emi_prob[row][col] = (edge_strength[row][col]/100) * max_search 
maxi=argmax(emi_prob[:,df.shape[1]-1])             

3.419811320754717e-05
(0, 1)
0.001440566037735849
(0, 1)
2.830188679245283e-05
(1, 1)


In [81]:
# maxi=argmax(emi_prob[:,df.shape[1]-1])

In [85]:
ridge_viterbi = zeros(df.shape[1])
for col in range(df.shape[1]-1,-1,-1):
    ridge_viterbi[col]=int(maxi)
    maxi=store_states[int(maxi)][col]
ridge_viterbi    

array([-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [83]:
imageio.imwrite("output_G.jpg", draw_edge(input_image, ridge_viterbi, (0, 255, 0), 5))